In [ ]:
import findspark
findspark.init()
import pyspark as spark
from pyspark.sql.functions import udf
import pyspark.sql.functions as F
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
sc = spark.SparkContext()
sql = spark.SQLContext(sc)

In [ ]:
level = 1
label='primary_site'
L = 3
labels = ['primary_site', 'disease_type', 'disease_tissue']
#labels = ['primary_site', 'secondary_site']
algorithm = 'topsbm'
#labels = ['primary_site', 'secondary_site', 'status']
#labels=['RPPA Clusters']
directory = "/home/fvalle/phd/results/tcga/oversampling_10tissue"
#directory = "/Users/filippo/Google Drive File Stream/My Drive/tesi_magistrale/tesi/results/hSBM/highlyvariable_7tissues"
os.chdir(directory)

In [ ]:
df_Pwt = sql.read.option('header',True).csv("%s/%s/%s_level_%d_word-dist.csv"%(directory,algorithm,algorithm,L))

In [ ]:
for topic in df_Pwt.columns[:5]:
    dist = df_Pwt.select([topic]).filter(F.col(topic)>1e-5).sort([topic], ascending=False).toPandas().astype(float).values
    if len(dist)<2:
        continue
    plt.title(topic)
    plt.plot(np.sort(dist[dist>0].T)[::-1], marker='x')
    plt.ylabel("$P(word | topic$)", fontsize=20)
    plt.xlabel("words", fontsize=20)
    plt.xscale('log')
    plt.yscale('log')
    plt.show()

In [ ]:
df_Ptd = sql.read.option('header',True).csv("%s/%s/%s_level_%d_topic-dist.csv"%(directory,algorithm,algorithm,L)).drop('i_doc')

In [ ]:
plt.figure()
plt.ylabel("$P(topic | sample$)", fontsize=20)
plt.xlabel("topics", fontsize=20)
for sample in df_Ptd.select('doc').toPandas().values[:10]:
    plt.plot(np.sort(df_Ptd.filter(F.col('doc')==sample[0]).drop('doc').toPandas().astype(float).values[0])[::-1])
plt.xscale('log')
plt.yscale('log')
plt.show()

In [ ]:
import tensorflow as tf

In [ ]:
Pgt = tf.convert_to_tensor(df_Pwt.toPandas().set_index('_c0').astype(float).values)
Pts = tf.transpose(tf.convert_to_tensor(df_Ptd.toPandas().set_index('doc').astype(float).values))

In [ ]:
Pgt

In [ ]:
Pts

In [ ]:
with tf.Session() as sess:
    Pgs = sess.run(tf.matmul(Pgt,Pts))
Pgs.shape

In [ ]:
df_Pgs = pd.DataFrame(data=Pgs, index=df_Pwt.select(['_c0']).toPandas().values.T[0], columns=df_Ptd.select(['doc']).toPandas().values.T[0])

In [ ]:
for doc in df_Pgs.columns[:5]:
    #doc = df_Pgs.columns[0]
    cdf = df_Pgs[doc]
    s_values = cdf[cdf>1e-95].sort_values(ascending=False)
    s_values = s_values/np.sum(s_values)
    plt.plot(s_values)
    plt.ylabel("$P(word | sample$)", fontsize=20)
    plt.xlabel("words", fontsize=20)
    plt.xscale('log')
    plt.yscale('log')
    plt.show()

In [ ]:
s_values_hsbm=s_values

In [ ]:
df_mt = sql.read.option("header",True).csv("%s/mainTable.csv"%directory)

In [ ]:
abundancies = df_mt.toPandas().set_index('_c0').astype(int).sum(axis=1)

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(np.sort(abundancies.values)[::-1]/np.sum(abundancies), label='avg')
plt.plot(s_values_hsbm.values, label='hsbm')
plt.plot(s_values_lda.values, label='lda')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.show()

In [ ]:
s_values_hsbm.values

In [ ]:
s_values_lda.values

In [ ]:
s_values